https://mauricio.github.io/2013/11/25/learning-scala-by-building-scala-lists.html   
https://mauricio.github.io/2013/12/08/learning-scala-by-building-scala-lists-part-2.html

In [1]:
sealed trait List[+A]
case object Nil extends List[Nothing]
case class Cons[+A](head: A, tail: List[A]) extends List[A]

In [2]:
object List {
    def apply[A](as: A*): List[A] = {
        if(as.isEmpty) Nil
        else Cons(as.head, apply(as.tail: _*))
    }
    
    def sum(list: List[Int]): Int = list match {
        case Nil => 0
        case Cons(head, tail) => head + sum(tail)
    }
    
}

### Variadic functions in Scala

The function apply in the object List is a variadic function, meaning it accepts zero
or more arguments of type A:

```scala
def apply[A](as: A*): List[A] =
    if (as.isEmpty) Nil
    else Cons(as.head, apply(as.tail: _*))
```

Variadic functions are just providing a little syntactic sugar for creating and passing
a Seq of elements explicitly. 

The special _* type annotation allows us to pass a Seq to a
variadic method.

In [3]:
val x = List(1,2,3,4,5) match {
    case Cons(x, Cons(2, Cons(4, _))) => x
    case Nil => 42
    case Cons(x, Cons(y, Cons(3, Cons(4, _)))) => x + y //This gets selected!
    case Cons(h, t) => h + List.sum(t)
    case _ => 101
}
x

3

In [4]:
def setHead[A](value: A, list: List[A]): List[A] =  list match {
    case Nil => Cons(value, Nil)
    case Cons(head, tail) => Cons(value, tail)
}

In [5]:
setHead(6, List(1,2,3,4,5))

Cons(6,Cons(2,Cons(3,Cons(4,Cons(5,Nil)))))

In [6]:
def drop[A](list: List[A]): List[A] = list match {
    case Nil => Nil
    case Cons(head, tail) => tail
}

In [7]:
drop(List(1,2,3,4,5))

Cons(2,Cons(3,Cons(4,Cons(5,Nil))))

In [8]:
//Implement dropWhile, which removes elements from the List prefix as long as they
// match a predicate.
def dropWhile[A](l: List[A], f: A => Boolean): List[A] = l match {
    case Nil => Nil
    case Cons(head: A, tail: List[A]) => 
      if(f(head)) dropWhile(tail, f) else Cons(head, dropWhile(tail, f))
}

In [9]:
dropWhile(List(1,2,3,4,5,6,7,8), (x:Int) => x%2 == 0)

Cons(1,Cons(3,Cons(5,Cons(7,Nil))))

In [10]:
def append[A](a1: List[A], a2: List[A]): List[A] =
    a1 match {
        case Nil => a2
        case Cons(h,t) => Cons(h, append(t, a2))
    }

In [11]:
def init[A](l: List[A]): List[A] = l match {
    case Nil => Nil
    case Cons(head, Nil) => Nil
    case Cons(head, tail) => Cons(head, init(tail) )
}

init(List(1,2,3,4,5))

Cons(1,Cons(2,Cons(3,Cons(4,Nil))))

### Improving type inference for higher-order functions1

In [12]:
def dropWhileCurried[A](l: List[A])(f: A => Boolean): List[A] = l match {
    case Nil => Nil
    case Cons(head: A, tail: List[A]) => 
      if(f(head)) dropWhile(tail, f) else Cons(head, dropWhile(tail, f))
}

dropWhileCurried(List(1,2,3,4,5,6,7,8))(x => x%2 != 0)

Cons(2,Cons(4,Cons(6,Cons(8,Nil))))

One way of describing what foldRight does is that it replaces the constructors of the list, Nil and Cons, with z and f, illustrated here:
```scala
Cons(1, Cons(2, Nil))
f   (1, f   (2, z ) )


foldRight(Cons(1, Cons(2, Cons(3, Nil))), 0)((x,y) => x + y)
1 + foldRight(Cons(2, Cons(3, Nil)), 0)((x,y) => x + y)
1 + (2 + foldRight(Cons(3, Nil), 0)((x,y) => x + y))
1 + (2 + (3 + (foldRight(Nil, 0)((x,y) => x + y))))
1 + (2 + (3 + (0)))
6

```

In [46]:
def foldRight[A,B](as: List[A], z: B)(f: (A,B) => B): B = as match {
    case Nil => z
    case Cons(head, tail) => f(head, foldRight(tail, z)(f))
}

In [14]:
def sum(ns: List[Int]) = {
    foldRight(ns, 0)(_+_)
}

sum(List(1,2,3,4,5))

15

In [15]:
foldRight(List(1,2,3), Nil:List[Int])(Cons(_,_))

Cons(1,Cons(2,Cons(3,Nil)))

In [25]:
def length[A](as: List[A]): Int = {
    foldRight(as, 0)((x, z) => 1+z)
}
length(List(1,2,3,4,5,6,23))

7

In [19]:
def foldLeft[A,B](as: List[A], z: B)(f: (B, A) => B): B = as match {
    case Nil => z
    case Cons(head, tail) => foldLeft(tail, f(z, head))(f)
}

TODO manual

In [22]:
def product(ns: List[Int]) = {
    foldLeft(ns, 1)(_*_)
}

product(List(1,2,3,4,5))

120

In [23]:
def sum(ns: List[Int]) = {
    foldLeft(ns, 0)(_+_)
}

sum(List(1,2,3,4,5))

15

In [26]:
def length[A](as: List[A]): Int = {
    foldLeft(as, 0)((z, x) => 1+z)
}
length(List(1,2,3,4,5,6,23))

7

In [48]:
def reverse(as: List[A]) =  {
    foldLeft(as, Nil)(Cons(_,_))
}

reverse(List(1,2,3,4,5))

Name: Compile Error
Message: <console>:32: error: not found: type A
       def reverse(as: List[A]) =  {
                            ^
<console>:33: error: type mismatch;
 found   : Cons[Nil.type]
 required: Nil.type
           foldLeft(as, Nil)(Cons(_,_))
                                 ^
StackTrace: 

In [31]:
def foldRight[A,B](as: List[A], z: B)(f: (A, B)=>B) = {
    foldLeft(as, z)((b,a) => f(a,b))
}

In [32]:
def sum(ns: List[Int]) = {
    foldRight(ns, 0)(_+_)
}

sum(List(1,2,3,4,5))

15

In [47]:
def append[A](as: List[A], value: A) = {
    foldRight(as, Cons(value, Nil))(Cons(_,_))
}

append(List(1,2,3,4,5),6)

Cons(1,Cons(2,Cons(3,Cons(4,Cons(5,Cons(6,Nil))))))

Hard: Write a function that concatenates a list of lists into a single list. Its runtime
should be linear in the total length of all lists. Try to use functions we have already
defined.

In [39]:
// Write a function that transforms a list of integers by adding 1 to each element.
// (Reminder: this should be a pure function that returns a new List!)

def addOne(as: List[Int]) = {
    foldLeft(as, 1)((z,x) => x+1)
}

addOne(List(1,2,3,4,5))
        

6

In [ ]:
// Write a function that turns each value in a List[Double] into a String. You can use
// the expression d.toString to convert some d: Double to a String.


In [ ]:
// Write a function map that generalizes modifying each element in a list while maintaining the structure of the list. Here is its signature:12
// def map[A,B](as: List[A])(f: A => B): List[B]


In [ ]:
// def filter[A](as: List[A])(f: A => Boolean): List[A]
